In [1]:
import importlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [2]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [3]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

In [4]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [5]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

In [6]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [7]:
train_no_nan = train.fillna(-100)
test_no_nan = test.fillna(-100)

In [8]:
# we don't know if the test set has a temporal split, so we'll just try a random split for now
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [11]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FactorAnalysis
X, y = load_digits(return_X_y=True)
transformer = FactorAnalysis(n_components=7, random_state=0)
X_transformed = transformer.fit_transform(X)
X_transformed.shape

(1797, 7)

In [14]:
import torch
import torch.nn as nn
import second_neural_network
importlib.reload(second_neural_network)
model = second_neural_network.second_neural_network()

In [15]:
import get_average_val_err

importlib.reload(get_average_val_err)

print(get_average_val_err.get_val_err(3, train_no_nan, model))

592380
Fold  1  of  3  test indices: [     0      1      2 ... 197457 197458 197459]
len(val_index_list):  197460
Train Epoch: 1  Loss: 0.5909
Train Epoch: 2  Loss: 0.5383
Train Epoch: 3  Loss: 0.6674
Train Epoch: 4  Loss: 0.4663
Train Epoch: 5  Loss: 0.6502
validation error:  0.6590062098971942
Fold  2  of  3  test indices: [197460 197461 197462 ... 394917 394918 394919]
len(val_index_list):  197460
Train Epoch: 1  Loss: 0.5852
Train Epoch: 2  Loss: 0.4031
Train Epoch: 3  Loss: 0.6769
Train Epoch: 4  Loss: 0.6574
Train Epoch: 5  Loss: 0.7589
validation error:  0.663256464614485
Fold  3  of  3  test indices: [394920 394921 394922 ... 592377 592378 592379]
len(val_index_list):  197460
Train Epoch: 1  Loss: 0.4901
Train Epoch: 2  Loss: 0.4033
Train Epoch: 3  Loss: 0.7459
Train Epoch: 4  Loss: 0.8181
Train Epoch: 5  Loss: 0.5539
validation error:  0.6532735166987764
(0.6585120637368186, 1.673196506186521e-05, [0.6590062098971942, 0.663256464614485, 0.6532735166987764])


In [23]:
test["Predicted"] = model.predict(test_no_nan.drop(["id","date","y"],axis=1))
test[["id","Predicted"]].to_csv("submission_test_second_neural_network1.csv",index=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
